# Implementation of a convolutional RBM using Theano

This notebook implements an efficient cRBM with various training procedures and different layers.

## Part 0: Importing theano, Biopython and all the other useful tools

In [1]:
import theano
import theano.tensor as T
import theano.tensor.nnet.conv as conv


import numpy as np
import Bio.SeqIO as sio
import Bio.motifs.matrix as mat
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio import motifs

import random
import time

ERROR (theano.sandbox.cuda): Failed to compile cuda_ndarray.cu: libcublas.so.7.0: cannot open shared object file: No such file or directory
ERROR:theano.sandbox.cuda:Failed to compile cuda_ndarray.cu: libcublas.so.7.0: cannot open shared object file: No such file or directory
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: cuda unavilable)


## Part 1: Getting and transforming the data

In [2]:
def readFASTASequencesFromFile (filename):
    dhsSequences = []
    for dhs in sio.parse(open(filename), 'fasta', IUPAC.unambiguous_dna):
        dhsSequences.append(dhs.seq)
    return dhsSequences

def readPWMsFromFile (filename):
    matrices = []
    for mat in motifs.parse(open(filename), 'jaspar'):
        matrices.append(mat.pwm)
    return matrices

### Read in FASTA sequences
These sequences are DNase1-hypersensitivity sites where each sample represents one DHS.
They were all already brought to the same dimensionality.

In [3]:
allSeqs = readFASTASequencesFromFile('data/wgEncodeAwgDnaseUwAg10803UniPk.fa')

In [181]:
pwms = readPWMsFromFile('data/jaspar_matrices.txt')

### Create a test set
Since working with these huge amounts of sequences is very time consuming, just start with a selection of them.

In [4]:
test_set = [allSeqs[random.randrange(0,len(allSeqs))] for i in range(1000)]
print "Length of test set: "  + str(len(test_set))

Length of test set: 1000


### Convert test set sequences to matrices we can work with
Each sequence is converted to a matrix of dimensionality (2 x 4 x N_v), representing:
* the two strands (first dimension)
* the four letters of the DNA alphabet (second dimension)
* the sequence represented by 1 (letter is present at this position) or 0 (letter is not present) (third dimension)

In [37]:
def getIntToLetter (letter):
    if letter == 'A' or letter == 'a':
        return 0
    elif letter == 'C' or letter == 'c':
        return 1
    elif letter == 'G' or letter == 'g':
        return 2
    elif letter == 'T' or letter == 't':
        return 3
    else:
        print "ERROR. LETTER " + letter + " DOES NOT EXIST!"
        return -1

def getMatrixFromSeq (seq):
    m = len(seq.alphabet.letters)
    n = len(seq)
    result = np.zeros((2, m, n))
    for i in range(len(seq)):
        result[0,getIntToLetter(seq[i]),i] = 1
        result[1,getIntToLetter(seq[i]),i] = 1
    return result

In [38]:
start = time.time()
dataMat = np.array([getMatrixFromSeq(t) for t in test_set])
print "Conversion of sequence to matrix for the test set in (in ms): " + str((time.time()-start)*1000)

Conversion of sequence to matrix for the test set in (in ms): 308.15911293


## Part 2a: Borrowing Ian Goodfellow's implementation of the prob. max pooling layer
This implementation is now part of the pylearn2 library which is licensed under the 3-claused BSD license.
It should not be a big issue using this as long as we cite the pylearn2 guys properly.

Source code is available here: https://github.com/lisa-lab/pylearn2/blob/master/pylearn2/expr/probabilistic_max_pooling.py

In [7]:
from theano.gof.op import get_debug_values

def max_pool(z, pool_shape, top_down=None, theano_rng=None):
    """
    Probabilistic max-pooling
    Parameters
    ----------
    z : theano 4-tensor
        a theano 4-tensor representing input from below
    pool_shape : tuple
        tuple of ints. the shape of regions to be pooled
    top_down : theano 4-tensor, optional
        a theano 4-tensor representing input from above
        if None, assumes top-down input is 0
    theano_rng : MRG_RandomStreams, optional
        Used for random numbers for sampling
    Returns
    -------
    p : theano 4-tensor
        the expected value of the pooling layer p
    h : theano 4-tensor
        the expected value of the detector layer h
    p_samples : theano 4-tensor, only returned if theano_rng is not None
        samples of the pooling layer
    h_samples : theano 4-tensor, only returned if theano_rng is not None
        samples of the detector layer
    Notes
    ------
    all 4-tensors are formatted with axes ('b', 'c', 0, 1).
    This is for maximum speed when using theano's conv2d
    to generate z and top_down, or when using it to infer conditionals of
    other layers using the return values.
    Detailed description:
    Suppose you have a variable h that lives in a Conv2DSpace h_space and
    you want to pool it down to a variable p that lives in a smaller
    Conv2DSpace p.
    This function does that, using non-overlapping pools.
    Specifically, consider one channel of h. h must have a height that is a
    multiple of pool_shape[0] and a width that is a multiple of pool_shape[1].
    A channel of h can thus be broken down into non-overlapping rectangles
    of shape pool_shape.
    Now consider one rectangular pooled region within one channel of h.
    I now use 'h' to refer just to this rectangle, and 'p' to refer to
    just the one pooling unit associated with that rectangle.
    We assume that the space that h and p live in is constrained such
    that h and p are both binary and p = max(h). To reduce the state-space
    in order to make probabilistic computations cheaper we also
    constrain sum(h) <= 1.
    Suppose h contains k different units. Suppose that the only term
    in the model's energy function involving h is -(z*h).sum()
    (elemwise multiplication) and the only term in
    the model's energy function involving p is -(top_down*p).sum().
    Then P(h[i] = 1) = softmax( [ z[1], z[2], ..., z[k], -top_down] )[i]
    and P(p = 1) = 1-softmax( [z[1], z[2], ..., z[k], -top_down])[k]
    This variation of the function assumes that z, top_down, and all
    return values use Conv2D axes ('b', 'c', 0, 1).
    This variation of the function implements the softmax using a
    theano graph of exp, maximum, sub, and div operations.
    Performance notes:
    It might be possible to make a faster implementation with different
    theano ops. rather than using set_subtensor, it might be possible
    to use the stuff in theano.sandbox.neighbours. Probably not possible,
    or at least nasty, because that code isn't written with multiple
    channels in mind, and I don't think just a reshape can fix it.
    Some work on this in galatea.cond.neighbs.py
    At some point images2neighbs' gradient was broken so check that
    it has been fixed before sinking too much time into this.
    Stabilizing the softmax is also another source of slowness.
    Here it is stabilized with several calls to maximum and sub.
    It might also be possible to stabilize it with
    T.maximum(-top_down,T.signal.downsample.max_pool(z)).
    Don't know if that would be faster or slower.
    Elsewhere in this file I implemented the softmax with a reshape
    and call to Softmax / SoftmaxWithBias.
    This is slower, even though Softmax is faster on the GPU than the
    equivalent max/sub/exp/div graph. Maybe the reshape is too expensive.
    Benchmarks show that most of the time is spent in GpuIncSubtensor
    when running on gpu. So it is mostly that which needs a faster
    implementation. One other way to implement this would be with
    a linear.Conv2D.lmul_T, where the convolution stride is equal to
    the pool width, and the thing to multiply with is the hparts stacked
    along the channel axis. Unfortunately, conv2D doesn't work right
    with stride > 2 and is pretty slow for stride 2. Conv3D is used to
    mitigate some of this, but only has CPU code.
    """

    z_name = z.name
    if z_name is None:
        z_name = 'anon_z'

    batch_size, ch, zr, zc = z.shape

    r, c = pool_shape

    zpart = []

    mx = None

    if top_down is None:
        t = 0.
    else:
        t = - top_down
        t.name = 'neg_top_down'

    for i in xrange(r):
        zpart.append([])
        for j in xrange(c):
            cur_part = z[:, :, i:zr:r, j:zc:c]
            if z_name is not None:
                cur_part.name = z_name + '[%d,%d]' % (i, j)
            zpart[i].append(cur_part)
            if mx is None:
                mx = T.maximum(t, cur_part)
                if cur_part.name is not None:
                    mx.name = 'max(-top_down,' + cur_part.name + ')'
            else:
                max_name = None
                if cur_part.name is not None:
                    mx_name = 'max(' + cur_part.name + ',' + mx.name + ')'
                mx = T.maximum(mx, cur_part)
                mx.name = mx_name
    mx.name = 'local_max(' + z_name + ')'

    pt = []

    for i in xrange(r):
        pt.append([])
        for j in xrange(c):
            z_ij = zpart[i][j]
            safe = z_ij - mx
            safe.name = 'safe_z(%s)' % z_ij.name
            cur_pt = T.exp(safe)
            cur_pt.name = 'pt(%s)' % z_ij.name
            pt[-1].append(cur_pt)

    off_pt = T.exp(t - mx)
    off_pt.name = 'p_tilde_off(%s)' % z_name
    denom = off_pt

    for i in xrange(r):
        for j in xrange(c):
            denom = denom + pt[i][j]
    denom.name = 'denom(%s)' % z_name

    off_prob = off_pt / denom
    p = 1. - off_prob
    p.name = 'p(%s)' % z_name

    hpart = []
    for i in xrange(r):
        hpart.append([pt_ij / denom for pt_ij in pt[i]])

    h = T.alloc(0., batch_size, ch, zr, zc)

    for i in xrange(r):
        for j in xrange(c):
            h.name = 'h_interm'
            h = T.set_subtensor(h[:, :, i:zr:r, j:zc:c], hpart[i][j])

    h.name = 'h(%s)' % z_name

    if theano_rng is None:
        return p, h
    
    ### --------------------- DONE IF NO SAMPLES ARE GENERATED ---------------------------###
    else:
        events = []
        for i in xrange(r):
            for j in xrange(c):
                events.append(hpart[i][j])
        events.append(off_prob)

        events = [event.dimshuffle(0, 1, 2, 3, 'x') for event in events]

        events = tuple(events)

        stacked_events = T.concatenate(events, axis=4)

        rows = zr // pool_shape[0]
        cols = zc // pool_shape[1]
        outcomes = pool_shape[0] * pool_shape[1] + 1
        assert stacked_events.ndim == 5
        for se, bs, r, c, chv in get_debug_values(stacked_events, batch_size,
                                                  rows, cols, ch):
            assert se.shape[0] == bs
            assert se.shape[1] == r
            assert se.shape[2] == c
            assert se.shape[3] == chv
            assert se.shape[4] == outcomes
        reshaped_events = stacked_events.reshape((
            batch_size * rows * cols * ch, outcomes))

        multinomial = theano_rng.multinomial(pvals=reshaped_events,
                                             dtype=p.dtype)

        reshaped_multinomial = multinomial.reshape((batch_size, ch, rows,
                                                    cols, outcomes))

        h_sample = T.alloc(0., batch_size, ch, zr, zc)

        idx = 0
        for i in xrange(r):
            for j in xrange(c):
                h_sample = T.set_subtensor(h_sample[:, :, i:zr:r, j:zc:c],
                                           reshaped_multinomial[:, :, :, :,
                                           idx])
                idx += 1

        p_sample = 1 - reshaped_multinomial[:, :, :, :, -1]

        return p, h, p_sample, h_sample


## Part 3: Optimizing theano to do it all on the GPU
We don't want only the convolutions to happen on the GPU because that means that every time we perform a convolution, the whole information is transferred to the GPU and back after it.

In [47]:
from theano.tensor.shared_randomstreams import RandomStreams


class CRBM:

    def __init__ (self, _motifLength, _numMotifs, _learningRate=0.1, _poolingFactor=1, _alphabet=IUPAC.unambiguous_dna):
        # parameters for the motifs
        self.motifLength = _motifLength
        self.numMotifs = _numMotifs
        self.alphabet = _alphabet
        self.initializeMotifs()
        
        # cRBM parameters
        self.bias = theano.shared(value=np.random.rand(self.numMotifs), name='bias', borrow=True)
        self.c = theano.shared(value=np.array([random.random()]), name='c', borrow=True)
        self.poolingFactor = _poolingFactor
        self.learningRate = _learningRate
        
        # infrastructural parameters
        self.rng = np.random.RandomState()
        self.theano_rng = RandomStreams(self.rng.randint(2**30))
    
    
    def initializeMotifs (self):
        # create random kernel
        x = np.random.rand(self.numMotifs, 2, 4, self.motifLength).astype(np.float32)
        
        # create reverse complement
        for i in range(self.numMotifs):
            x[i,1] = np.flipud(np.fliplr(x[i,0]))
            
        self.motifs = theano.shared(value=x, name='W', borrow=True)
        
        
    def setCustomKernels (self, customKernels):
        if len(customKernels.shape) != 4:
            print "New motifs must be a 4D matrix with dims: (K x numOfStrands(2) x numOfLetters(4) x numOfKMers)"
            return
        
        self.numMotifs = customKernels.shape[0]
        self.motifLength = customKernels.shape[3]
        self.bias = theano.shared(value=np.random.rand(self.numMotifs), name='bias', borrow=True)
        self.motifs = theano.shared(value=customKernels.astype(np.float32))
        print "New motifs set. # Motifs: " + str(self.numMotifs) + " K-mer-Length: " + str(self.motifLength)

        
### ------------------------------THE TOUGH STUFF-------------------------------- ###
### ----------------------------------------------------------------------------- ###

    def forwardBatch (self, data):
        out = conv.conv2d(data, self.motifs)[:,:,::-1,::-1] # flip, because conv reverts H
        bMod = self.bias.dimshuffle('x', 0, 'x', 'x') # add dims to the bias until it works
        pooled = max_pool(out + bMod, pool_shape=(1, self.poolingFactor), theano_rng=self.theano_rng)
        return [pooled[1], pooled[3]] #only return pooled layer and probs


    def backwardBatch (self, H_sample):
        # theano convolution call
        #K_star = self.motifs.dimshuffle(1, 0, 2, 3)[:,:,::-1,::-1]
        C = conv.conv2d(H_sample, self.motifs[:,:,:,::-1], border_mode='full')
        out = T.sum(C, axis=1) # sum over all K
        #out = out + self.c
        
        # add fourth dimension (the strands) that was lost during forward pass
        res = T.tile(self.softmax(out).dimshuffle(0,'x',1,2),[1,2,1,1])
        return res


    def gradient (self, hiddenProbs, data):
        mean = T.mean(hiddenProbs, axis=0, keepdims=True) # sum over all samples to get avg (but keep dim)
        mean = T.tile(mean, [2,1,1,1])
        H_reshaped = mean.dimshuffle(1, 0, 2, 3)
        out = conv.conv2d(data, H_reshaped)
        return T.mean(out, axis=0)

    
    def batchTraining (self, data, epochs, batchSize, numOfCDs):
        
        # calculate the data gradient for weights (motifs) and bias
        D_batch = T.tensor4('data')
        [H_data, S_data] = self.forwardBatch(D_batch)
        G_data = self.gradient(H_data, D_batch)
        bias_data = T.mean(T.sum(H_data, axis=3), axis=0)
        
        # calculate the model gradient (only CD-1 for now)
        V_model = self.backwardBatch(S_data)
        S_v = self.sampleVisibleLayer(V_model)
        [H_model, S_model] = self.forwardBatch(S_v)
        G_model = self.gradient(H_model, D_batch)
        bias_model = T.mean(T.sum(H_model, axis=3), axis=0)
        
        self.motifs = self.motifs + self.learningRate * (G_data - G_model)
        self.bias = self.bias + self.learningRate * (bias_data - bias_model)
        
        training = theano.function([D_batch], [self.motifs,self.bias], allow_input_downcast=True)
        
        print "START"
        
        # after we got the theano function for everything, let's apply it!
        itPerEpoch = data.shape[0] / batchSize
        for epoch in range(epochs):
            for batch in range(itPerEpoch):
                print batch*batchSize, (batch+1)*batchSize
                training = training(data[batch*batchSize:(batch+1)*batchSize])
            print "Epoch done: " + str(epoch)
        
        return training
        
        
        
 
    def sampleVisibleLayer (self, V):
        S = self.theano_rng.multinomial(n=1,pvals=V.dimshuffle(0,1,3,2)).dimshuffle(0,1,3,2)
        S = S.astype('float32')
        return S
    
    def softmax (self, x):
        return T.exp(x) / T.exp(x).sum(axis=1, keepdims=True)


In [50]:
theano.config.exception_verbosity='high'
theano.config.optimizer='None'
theano.config.compute_test_value='ignore'

x = CRBM(4,2,0.1, 3)
kernel1 = np.tile(np.array([[1,0,0],[0,1,0],[0,0,1],[0,0,0]]), [2,1,1])
kernel2 = np.tile(np.array([[0,0,0],[0,0,0],[1,1,1],[0,0,0]]), [2,1,1])
#kernel1 = np.tile(np.eye(4), [2,1,1])
#kernel2 = np.tile(np.array([[0,0,0,0],[0,0,0,0],[1,1,1,1],[0,0,0,0]]), [2,1,1])
kernel = np.array([kernel1, kernel2])
filter_shape = kernel.shape

randSeq1 = getMatrixFromSeq(Seq("ACGTGGGG", IUPAC.unambiguous_dna))
randSeq2 = getMatrixFromSeq(Seq("ACGTACGT", IUPAC.unambiguous_dna))
data = np.array([randSeq1, randSeq2])
print "Data shape: " + str(data.shape)
#x.setCustomKernels(kernel)




print "Data mat shape: " + str(dataMat.shape)
res = x.batchTraining(data, 1, 1, 10)
print "Result from training: "
print res[0]
print res[1]

Data shape: (2, 2, 4, 8)
Data mat shape: (1000, 2, 4, 150)
START
0 1


ValueError: Input dimension mis-match. (input[0].shape[3] = 2, input[1].shape[3] = 1)
Apply node that caused the error: Elemwise{maximum,no_inplace}(max(anon_z[0,1],max(-top_down,anon_z[0,0])), anon_z[0,2])
Inputs types: [TensorType(float64, 4D), TensorType(float64, 4D)]
Inputs shapes: [(1, 2, 1, 2), (1, 2, 1, 1)]
Inputs strides: [(32, 16, 16, 8), (80, 40, 40, 24)]
Inputs values: [array([[[[ 3.39141388,  3.36073847]],

        [[ 5.65961594,  6.90420098]]]]), array([[[[ 3.77698393]],

        [[ 5.4668097 ]]]])]

Backtrace when the node is created:
  File "<ipython-input-7-93a9be5a9de2>", line 120, in max_pool
    mx = T.maximum(mx, cur_part)

Debugprint of the apply node: 
Elemwise{maximum,no_inplace} [@A] <TensorType(float64, 4D)> 'local_max(anon_z)'   

Storage map footprint:
 - W, Shape: (2, 2, 4, 4), ElemSize: 4 Byte(s), TotalSize: 256 Byte(s)
 - Constant{3}, Shape: (1,), ElemSize: 8 Byte(s), TotalSize: 8.0 Byte(s)
 - Constant{-1}, Shape: (1,), ElemSize: 8 Byte(s), TotalSize: 8.0 

In [ ]:
# Construct a kernel matrix, looking for ACGT and GGGG
kernel1 = np.tile(np.eye(4), [2,1,1]) # the kernel will only look for ACGT
kernel2 = np.tile(np.array([[0,0,0,0],[0,0,0,0],[1,1,1,1],[0,0,0,0]]), [2,1,1]) # this one looks for GGGG
kernel2[1,:,:] = np.array([[0,0,0,0],[1,1,1,1],[0,0,0,0],[0,0,0,0]])
kernel3 = np.tile(np.zeros((4,4)), [2,1,1])
kernel = np.array([kernel1, kernel2, kernel3])
print kernel.shape
print x.motifs.shape
# create toy sequences where we might be able to see what happens
randSeq1 = getMatrixFromSeq(Seq("ACGTGGGG", IUPAC.unambiguous_dna))
randSeq2 = getMatrixFromSeq(Seq("ACGTACGT", IUPAC.unambiguous_dna))
data = np.array([randSeq1, randSeq2])

print data.shape
print dataMat.shape
# initialize the learner, insert our predefined kernels and train
cRBM = ConvRBM_slow (4, 3, 0.1, 1)
cRBM.setCustomKernels(kernel)
np.set_printoptions(precision=2, suppress=True)
cRBM.batchTraining(data, 10, 2, 1)
print cRBM.motifs

### Verify that the algorithm is doing something meaningful
We can now apply the learning algorithm on a toy sequence to see what happens.
Important questions to ask:

* Does the hidden layer somehow make sense?
* What does the sigmoid function do with it?
* How does the *reconstruction* look like?
* Is the softmax doing the right thing?

In [194]:
# Construct a kernel matrix, looking for ACGT and GGGG
kernel1 = np.tile(np.eye(4), [2,1,1]) # the kernel will only look for ACGT
kernel2 = np.tile(np.array([[0,0,0,0],[0,0,0,0],[1,1,1,1],[0,0,0,0]]), [2,1,1]) # this one looks for GGGG
kernel2[1,:,:] = np.array([[0,0,0,0],[1,1,1,1],[0,0,0,0],[0,0,0,0]])
kernel3 = np.tile(np.zeros((4,4)), [2,1,1])
print kernel3.shape
print kernel2.shape
kernel = np.array([kernel1, kernel2, kernel3])

# create toy sequences where we might be able to see what happens
randSeq1 = getMatrixFromSeq(Seq("ACGTGGGG", IUPAC.unambiguous_dna))
randSeq2 = getMatrixFromSeq(Seq("ACGTACGT", IUPAC.unambiguous_dna))
data = np.array([randSeq1, randSeq2])

# initialize the learner and insert our predefined kernels
cRBM = ConvRBM_slow (4, 3, 0.1, 5)
cRBM.setCustomKernels(kernel)
np.set_printoptions(precision=2, suppress=True)

res = cRBM.probMaxPooling(cRBM.forwardBatch(data))
print "Result from forward pass (that is: P(H | V))"
print "Shape: -> " + str(res.shape)
print res

vis = cRBM.softmax(cRBM.backwardBatch(res))
print "Result from backward pass (that is: P(V | H))"
print "Shape: -> " + str(vis.shape)
print vis
grad = cRBM.gradient(res, data)
print "Result from gradient calculation"
print "Shape: -> " + str(grad.shape)
print grad

(2, 4, 4)
(2, 4, 4)
New motifs set. # Motifs: 3 K-mer-Length: 4


ValueError: the filter stack size (2) and image stack size (1) differ
Apply node that caused the error: ConvOp{('imshp', (None, None, None)),('kshp', (None, None)),('nkern', None),('bsize', None),('dx', 1),('dy', 1),('out_mode', 'valid'),('unroll_batch', None),('unroll_kern', None),('unroll_patch', True),('imshp_logical', (None, None, None)),('kshp_logical', (None, None)),('kshp_logical_top_aligned', True)}(data, kernels)
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(2, 1, 4, 8), (3, 2, 4, 4)]
Inputs strides: [(128, 128, 32, 4), (128, 64, 16, 4)]
Inputs values: ['not shown', 'not shown']

Backtrace when the node is created:
  File "<ipython-input-189-8b88baef28e9>", line 82, in forwardBatch
    out = conv.conv2d(D,K)

HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

### Some performance tests on the test set
Now, let's test it on the whole test set. The performance obtained here, should tell us a lot on how fast we can actually train the learner.

In [195]:
cRBM = ConvRBM_slow (7, 10, 0.1, 3)
# perform forward and backward pass and calculate the gradient
start = time.time()
res = cRBM.probMaxPooling(cRBM.forwardBatch(dataMat))
vis = cRBM.softmax(cRBM.backwardBatch(res))
grad = cRBM.gradient(res, dataMat)

print res[0,0,:,:]
print "Forward pass: " + str(res.shape)
print "Backward pass: " + str(vis.shape)
print "Gradient: " + str(grad.shape)
print "Time for processing (for, back, grad) of " + str(dataMat.shape[0]) + " sequences: " + str((time.time()-start))

ValueError: the filter stack size (2) and image stack size (1) differ
Apply node that caused the error: ConvOp{('imshp', (None, None, None)),('kshp', (None, None)),('nkern', None),('bsize', None),('dx', 1),('dy', 1),('out_mode', 'valid'),('unroll_batch', None),('unroll_kern', None),('unroll_patch', True),('imshp_logical', (None, None, None)),('kshp_logical', (None, None)),('kshp_logical_top_aligned', True)}(data, kernels)
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(1000, 1, 4, 150), (10, 2, 4, 7)]
Inputs strides: [(2400, 2400, 600, 4), (224, 112, 28, 4)]
Inputs values: ['not shown', 'not shown']

Backtrace when the node is created:
  File "<ipython-input-189-8b88baef28e9>", line 82, in forwardBatch
    out = conv.conv2d(D,K)

HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [22]:
#del cRBM, test_set, allSeqs, pwms

## Part 2b: Implementing the cRBM
We now have all the tools we need to actually implement the cRBM.
Some notes on constraints and behavior of our learning algorithm:

* It uses Theano to do most (if not all) of the work. That means it's ways faster on a GPU (but not as fast as possible)
* It expects kernels (filters/motifs) to be of the same length. While this choice was made due to performance issues, it's not a problem because we expect the algorithm to combine multiple motifs if nessessary.
* The sequences (DHSes) are also expected to be of the same size

Now, finally some hints on usage:

* The pooling factor has to be evenly dividable by the length of the hidden units

In [ ]:
"""
This class implements a cRBM for sequence analysis.
It does perform efficient forward and backward pass of any given DNA sequence.
It implements softmax and sigmoid functions as activation and performs probabilistic max pooling
after the convolution step.
So this class is basically a two layer network with a convolution layer and a pooling layer on top
of that.
The learning procedure uses contrastive divergence (CD) with a variable amount of steps.
"""
class ConvRBM_slow:
    
    """
    Initialize the cRBM. The parameters here are global params that should not change
    during the execution of training or testing and characterize the network.
    
    Parameters:
    _motifLength:    How long are the motifs (position weight matrices PWM). This
                     This is equivalent to ask what the number of k-mers is.
                     The current approach only deals with one fixed motif length.
                     
    _numMotifs:      How many motifs are applied to the sequence, that is how many
                     hidden units does the network have. Each hidden unit consists
                     of a vector of size (sequenceLength-motifLength+1)
                     
    _poolingFactor:  How many units from the hidden layer are pooled together.
                     Note that the number has to divide evenly to the length of
                     the hidden units, that is:
                     mod(sequenceLength-motifLength+1, poolingFactor) == 0
                     (1 = equivalent to sigmoid activation)
                     
    _alphabet:       Biopython uses alphabets for sequences to do sanity checks.
                     However, all of the code is written for DNA sequences and even
                     though in theory there should be no difference between that
                     and other alphabets, Biopython may have trouble with the convolution.
    """
    def __init__ (self, _motifLength, _numMotifs, _learningRate=0.1, _poolingFactor=1, _alphabet=IUPAC.unambiguous_dna):
        # parameters for the motifs
        self.motifLength = _motifLength
        self.numMotifs = _numMotifs
        self.alphabet = _alphabet
        self.initializeMotifs()
        
        # cRBM parameters
        self.bias = np.random.rand(self.numMotifs)
        self.c = random.random()
        self.poolingFactor = _poolingFactor
        self.learningRate = _learningRate
        
        # infrastructural parameters
        self.rng = np.random.RandomState()
        
    
    
    def initializeMotifs (self):
        x = np.random.rand(self.numMotifs, 2, 4, self.motifLength)
        for i in range(self.numMotifs):
            x[i,1] = np.flipud(np.fliplr(x[i,0]))
        self.motifs = x
        
        
    """
    Calculate the forward pass for any given set of sequences, that is calculate P(H | V).
    This method applies convolution of all filters to all sequences in the set, using theano.
    It also looks on both strands for matches and returns the hidden activation layer.
    
    Parameters:
    data:            The DNA sequences to calculate the forward pass on. The data is a 4D matrix
                     with dimensionality: (N_batch x numOfStrands(2) x numOfLetters(4) x N_v)
                     
    Return:
    The function returns the hidden activation layer as numpy matrix.
    That matrix has dimensionality N_batch x K x 1 x N_h where
    K is the number of kernels (motifs/PWMs) that are applied and
    N_h is the length of the hidden layer (convolution is of type 'valid')
    
    Note that the strandness is lost during this process because it's added up in the hidden layer.
    """
    def forwardBatch (self, data):
        # create 4D tensor for theano (BatchSize x K x 2*numOfLetters x lenOfSeqs)
        D = T.tensor4('data')
        K = T.tensor4('kernels')
        out = conv.conv2d(D,K)
        f = theano.function([D,K], out, allow_input_downcast=True)
        
        bMod = self.bias[np.newaxis,:,np.newaxis,np.newaxis] # add dims to the bias until it works
        
        return f(data, self.motifs) + bMod



    """
    Calculates the backward pass on the data, that is P(V | H).
    It does so by performing convolution of the hidden layer and the motifs for each kernel using theano.
    
    Parameters:
    hidden layer:   The layer that was previously computed by the forward pass.
    
    Return:
    A matrix of dimensionality: N_batch x K x numOfLetters(4) x N_v
    The result can be interpreted as the probability for each position to have a specific letter
    once it is summed over all K (sum over second dimension).
    That means, the combination of summing over K and applying softmax can be interpreted as P(V | H).
    """
    def backwardBatch (self, hiddenActivation):
        # theano convolution call
        H = T.tensor4('hidden')
        K = T.tensor4('kernels')
        K_star = K.dimshuffle(1, 0, 2, 3)[:,:,::-1,::-1]
        C = conv.conv2d(H, K_star, border_mode='full')
        out = T.sum(C, axis=1) # sum over all K
        f = theano.function([H,K], out, allow_input_downcast=True)

        res = f(hiddenActivation, self.motifs) + self.c
        
        # add fourth dimension (the strands) that was lost during forward pass (max pooling)
        res = np.tile(res[:,np.newaxis,:,:], [1,2,1,1])
        return res



    """
    Set new kernels when you don't wish to start with random ones. Can be used if prior knowledge about
    the structure of the sequences is present.
    
    Parameters:
    customKernels:  A matrix of shape (K x numOfStrands(2) x numOfLetters(4) x num-of-k-mers) containing the
                    new kernels to work with.
                    
    Return:
    Nothing.
    """
    def setCustomKernels (self, customKernels):
        if len(customKernels.shape) != 4:
            print "New motifs must be a 4D matrix with dims: (K x numOfStrands(2) x numOfLetters(4) x numOfKMers)"
            return
        
        self.numMotifs = customKernels.shape[0]
        self.motifLength = customKernels.shape[3]
        self.bias = np.random.rand(self.numMotifs)
        self.motifs = customKernels
        print "New motifs set. # Motifs: " + str(self.numMotifs) + " K-mer-Length: " + str(self.motifLength)



    """
    Calculate the gradient for a given data matrix and samples from the hidden layer.
    The gradient is computed using theano and convolution.
    
    Parameters:
    hiddenProbs:    A matrix of shape (N_batch x K x numOfLetters(4) x N_v) containing probabilities
                    from the prob. distribution of the hidden layer.
                    Note that this matrix is obtained as result from forwardBatch().
                    
    data:           A matrix of shape (N_batch x numOfStrands(2) x numOfLetters(4) x N_v) containing
                    the data. Note that this matrix is usually the same as the matrix passed to forwardBatch().
                    
    Return:
    A matrix containing the gradient for all kernels/motifs/pwms.
    This matrix is of shape (N_batch x K x 1 x num-of-k-mers)
    """
    def gradient (self, hiddenProbs, data):
        H = T.tensor4('hidden')
        S = T.tensor4('sample')
        H_reshaped = H.dimshuffle(1, 0, 2, 3)
        out = conv.conv2d(S, H_reshaped)
        f = theano.function([H,S], out, allow_input_downcast=True)

        res = f(np.tile(np.mean(hiddenProbs, axis=0), [2,1,1,1]), np.tile(data, [1,1,1,1]))
        return np.mean(res, axis=0)



    def batchTraining (self, data, epochs, batchSize, numOfCDs):
        itPerEpoch = data.shape[0] / batchSize
        for epoch in range(epochs):
            for batch in range(itPerEpoch):
                D_batch = data[batch*batchSize:(batch+1)*batchSize]
                H = self.probMaxPooling(self.forwardBatch(D_batch))
                G_data = self.gradient(H, D_batch) # gradient for W for the data
                bias_data = np.mean(np.sum(H, axis=3), axis=0) # gradient for bias
                c_data = np.mean(np.sum(np.sum(np.sum(D_batch, axis=3), axis=2), axis=1), axis=0)
                print "C_data: " + str(c_data)
                for cd in range(numOfCDs):
                    S = self.sampleFromMatrix(H)
                    V = self.softmax(self.backwardBatch(S))
                    S_v = self.sampleFromMatrix(V)
                    H = self.probMaxPooling(self.forwardBatch(S_v))
                
                G_model = self.gradient(H, D_batch)
                bias_model = np.sum(np.sum(H, axis=3), axis=0)
                c_model = np.mean(np.sum(np.sum(np.sum(V, axis=3), axis=2), axis=1), axis=0)
                print "C_model: " + str(c_model)
                #print "Gradient of model: " + str((G_data-G_model).shape)
                #print "For data: " + str((batch*batchSize, (batch+1)*batchSize))
                #print G_data-G_model
                
                self.motifs = self.motifs + self.learningRate * (G_data - G_model)
                self.bias = self.bias + self.learningRate * (bias_data - bias_model).sum(axis=1)
                self.c = self.c + self.learningRate * (c_data - c_model)
                
            print "Epoch done: " + str(epoch)
        
        
        
    """
    Calculates the probabilistic max pooling layer (P) from a hidden layer H.
    P is sparse because only every poolingFactor'th unit can be active while all the other
    units are forced to zero. The one that becomes active is the one with the highest softmax
    activation from H.
    Parameters:
    H:              Hidden layer obtained by forwardBatch function.
    
    Return:
    A layer of the same shape as H, but sparsed out.
    """
    def probMaxPooling (self, H):

        # first of all some easy definitions
        n_h = H.shape[3]
        numOfGroups = n_h/self.poolingFactor
        #print "N_H = " + str(n_h) + " numOfGroups: " + str(numOfGroups) + " poolingFactor: " + str(self.poolingFactor)

        # exponentiate it all
        exp = np.exp(H)
        
        # get the right dimensions for the matrix
        dims = (exp.shape[0], exp.shape[1], self.poolingFactor, numOfGroups)
        reshaped = exp.reshape(dims)
        
        # append ones (to have the 5th, or non-active state), get the denominator and divide by it
        withOnes = np.insert(reshaped, self.poolingFactor, np.ones(numOfGroups), 2)
        denom = np.sum(withOnes, axis=2)
        div = withOnes / denom[:,:,np.newaxis,:]
        
        # now calculate argmax & max and insert into P
        P = np.zeros(div.shape)
        idx = np.argmax(div, axis=2)
        maxes = np.max(div, axis=2)
        
        # TODO: this is not performant!!!
        for sample in range(idx.shape[0]):
            for kernel in range(idx.shape[1]):
                for seqPos in range(idx.shape[2]):
                    P[sample,kernel,idx[sample,kernel,seqPos],seqPos] = maxes[sample, kernel, seqPos]

        # cut out the 5th extra dimension
        P = P[:,:,0:reshaped.shape[2],:]
        return np.reshape(P,(H.shape[0],H.shape[1],1,-1), 'F')



    """
    Maybe not working...
    """
    def sampleFromMatrix (self, M):
        boolean_mat = M > self.rng.random_sample(M.shape)
        return boolean_mat.astype(int)



    """
    Calculate the sigmoid activation for the hidden layer.
    """
    def sigmoid(self, _H):
        return (1.0 / (1.0 + np.exp(-_H)))



    """
    Calculates the softmax activation using numpy. This function was designed to work with the
    backward pass well.
    Using it for the probabilistic max pooling might not be recommended.
    Parameters:
    V:              The hidden layer calculated so far. That is, the result from convolution
                    when calling backwardBatch().
                    The matrix is of shape N_batch x K x numOfLetters(4) x N_v
    Return:
    A matrix of the same shape and size, but each column for the letters (3rd dim) has been softmaxed.
    """
    def softmax (self, V):
        exp = np.exp(-V) # exponentiate it all
        denominator = np.sum(exp, axis=1) # this the sum of all the exp(-x)
        # now expand the denominator such that the division works
        denominator = denominator[:,np.newaxis,:].repeat(exp.shape[1], axis=1)
        div = exp / denominator
        return div
